In [1]:
import pandas as pd
import numpy as np
import random as rd
import string as st
import matplotlib.pyplot as plt
from scipy.stats import rv_histogram
from scipy.stats import gaussian_kde
import seaborn as sns

/home/antonioskorin/.local/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# Elementos Duplicados

In [2]:
df = pd.read_csv("universities_followers.csv")
print(df.head())
print(f"Row count: {len(df)}\n\n")

# Convertimos desde notación científica
df["user_id"] = df["user_id"].astype(np.int64)
print(df.head())
print(f"Row count: {len(df)}")
df.to_csv("universities_followers_int64.csv", index = False)


df = pd.read_csv("universities_followers_int64.csv")

  university       user_id      user_name  number_tweets  friends_count  \
0    pucv_cl  7.585618e+17  SantillanaLAB           2293           1040   
1    pucv_cl  4.149421e+08  rodcontrabajo             20            154   
2    pucv_cl  6.821312e+07         ntejos            314            456   
3    pucv_cl  1.187383e+08        edu_hhs          13611           2650   
4    pucv_cl  1.260729e+18  DatosLab_UPLA              0            173   

   followers_count                      created_at  
0             2528  Thu Jul 28 07:16:49 +0000 2016  
1               53  Thu Nov 17 17:57:02 +0000 2011  
2              183  Sun Aug 23 19:14:41 +0000 2009  
3             1313  Mon Mar 01 16:15:26 +0000 2010  
4                4  Thu May 14 01:02:38 +0000 2020  
Row count: 21070


  university              user_id      user_name  number_tweets  \
0    pucv_cl   758561835897479040  SantillanaLAB           2293   
1    pucv_cl            414942137  rodcontrabajo             20   
2    pucv_c

In [3]:
df_pruned = df.drop_duplicates()
print(df_pruned.head())
print(f"Row count: {len(df_pruned)}")

  university              user_id      user_name  number_tweets  \
0    pucv_cl   758561835897479040  SantillanaLAB           2293   
1    pucv_cl            414942137  rodcontrabajo             20   
2    pucv_cl             68213121         ntejos            314   
3    pucv_cl            118738304        edu_hhs          13611   
4    pucv_cl  1260728998226800128  DatosLab_UPLA              0   

   friends_count  followers_count                      created_at  
0           1040             2528  Thu Jul 28 07:16:49 +0000 2016  
1            154               53  Thu Nov 17 17:57:02 +0000 2011  
2            456              183  Sun Aug 23 19:14:41 +0000 2009  
3           2650             1313  Mon Mar 01 16:15:26 +0000 2010  
4            173                4  Thu May 14 01:02:38 +0000 2020  
Row count: 21056


In [4]:
duplicate_ids = df_pruned["user_id"].duplicated()
duplicate_usernames = df_pruned["user_name"].duplicated()

print("Filas donde hay duplicados de id: ")
print(duplicate_ids[duplicate_ids].index)
print("Filas donde hay duplicados de user_name: ")
print(duplicate_usernames[duplicate_usernames].index)


print("\n\nCuentas de duplicados en total: ")
dup_id_count = duplicate_ids.value_counts()
dup_usernames_count = duplicate_usernames.value_counts()
print(dup_id_count)
print(dup_usernames_count)



Filas donde hay duplicados de id: 
Index([11071, 11075, 11081, 11083, 11087, 11091, 11093, 11104, 11108, 11124,
       ...
       20968, 20972, 20975, 20998, 21003, 21010, 21023, 21029, 21034, 21061],
      dtype='int64', length=1148)
Filas donde hay duplicados de user_name: 
Index([11071, 11075, 11081, 11083, 11087, 11091, 11093, 11104, 11108, 11124,
       ...
       20968, 20972, 20975, 20998, 21003, 21010, 21023, 21029, 21034, 21061],
      dtype='int64', length=1148)


Cuentas de duplicados en total: 
user_id
False    19908
True      1148
Name: count, dtype: int64
user_name
False    19908
True      1148
Name: count, dtype: int64


In [5]:
dups = set()
for idx in duplicate_usernames[duplicate_usernames].index:
    username = df_pruned["user_name"][idx]
    dups.add(username)
print("Cantidad de usuarios que se repiten: ")
print(len(dups))

Cantidad de usuarios que se repiten: 
838


In [6]:
df_pruned = df.drop_duplicates(subset=["user_id", "user_name"], keep="first")
# shuffleamos el dataset por si acaso
df_final = df_pruned.sample(frac=1).reset_index(drop=True)
print(df_final.head())

df_final.to_csv("universities_followers_no_dups.csv", index = False)

  university              user_id        user_name  number_tweets  \
0  UFrontera  1234648163694380032           CrspJv             30   
1      usach           1927636664  HombredeNoticia          20863   
2  UFrontera   713037885272776960    KrishMicrobio             28   
3    pucv_cl            368609388         KroRapel            249   
4    pucv_cl           3304979885  RodrigoVilche12             25   

   friends_count  followers_count                      created_at  
0            107                2  Tue Mar 03 01:14:09 +0000 2020  
1           1917              957  Wed Oct 02 17:43:58 +0000 2013  
2            210               35  Thu Mar 24 16:20:54 +0000 2016  
3            127               21  Mon Sep 05 22:49:11 +0000 2011  
4             51                8  Sun May 31 19:58:21 +0000 2015  


# Elementos Aleatorios

In [7]:
def random_username_forbidden(usr: str, forbidden: set) -> str:
    temp = usr
    #para debugear
    i = 0
    while temp in forbidden:
        if i > 0: 
            print(f"{temp}, generated from {usr} is Forbidden, trying again.")
        temp = random_username(usr)
    return temp



# Inspirado en distancia de Levenshtein, código original.
# Un usuario de twitter solo tiene letras, números, underscores, un máximo de 15 caracteres y un mínimo de 4.
def random_username(usr: str) -> str:
    n = len(usr)
    # Cuantos caracteres cambiar de la palabra
    n_edits = rd.randint(1, n)
    # Las operaciones que vamos a realizar, agrego letras ABC para sortear.
    ops = rd.choices(["C-sub", "B-in", "A-del"], k=n_edits)
    ops.sort()
    #print(f"Operaciones a realizar: {ops}")
    # Caracteres válidos para usuaro de twitter
    valid_chars = st.ascii_letters + st.digits + "_"

    # Haremos primero los replacements, después las inserciones y por último los delete.
    while ops:
        op = ops.pop()
        if op == "B-in":
            # Si insertar un caracter nos dejara sobre 15, cambiamos la operación una de las dos restantes.
            if len(usr) >= 15:
                op = rd.choice(["A-del", "C-sub"])
                ops.append(op)
                ops.sort()
                #print("Too long, making new choice.")
                continue
            else:
                # Podemos insertar donde sea, incluso en len(usr)
                idx = rd.randint(0, len(usr))
                ch = rd.choice(valid_chars)
                usr = usr[:idx] + ch + usr[idx:]
                #print(f"Insert '{ch}' at idx {idx}: " + usr)
        
        elif op == "A-del":
            # Si remover un caracter nos dejara con menos de 4, cambiamos la operación a una de las dos restantes.
            if len(usr) <= 4:
                op = op = rd.choice(["B-in", "C-sub"])
                ops.append(op)
                ops.sort()
                #print("Too short, making new choice.")
                continue
            else:
                idx = rd.randint(0, len(usr) - 1)
                usr = usr[:idx] + usr[idx + 1:]
                #print(f"Deleting idx {idx}: " + usr)

        elif op == "C-sub":
            idx = rd.randint(0, len(usr) - 1)
            ch = rd.choice(valid_chars)
            usr = usr[:idx] + ch + usr[idx + 1:]
            #print(f"Replacing idx {idx} for '{ch}': " + usr)        
    return usr


In [8]:
# Randomizamos usernames
random_usr_col = df_final["user_name"].apply(random_username_forbidden, forbidden=set(df_final["user_name"].unique()))
randomized_df = df_final.copy(deep=True)
randomized_df["user_name"] = random_usr_col
print(randomized_df.head())

  university              user_id       user_name  number_tweets  \
0  UFrontera  1234648163694380032         rsMKpJv             30   
1      usach           1927636664  ombredrNoticia          20863   
2  UFrontera   713037885272776960     k6KrisMiEoo             28   
3    pucv_cl            368609388        QrREp9el            249   
4    pucv_cl           3304979885    JMdibelchj12             25   

   friends_count  followers_count                      created_at  
0            107                2  Tue Mar 03 01:14:09 +0000 2020  
1           1917              957  Wed Oct 02 17:43:58 +0000 2013  
2            210               35  Thu Mar 24 16:20:54 +0000 2016  
3            127               21  Mon Sep 05 22:49:11 +0000 2011  
4             51                8  Sun May 31 19:58:21 +0000 2015  


In [9]:
user_ids = randomized_df["user_id"].values
user_ids_set = set(randomized_df["user_id"].unique())

hist, bin_edges = np.histogram(user_ids, bins=100, density=True)
hist_dist = rv_histogram((hist, bin_edges))

random_ids = list()
# Esto es medio desagradable
for i in range(randomized_df.shape[0]):
    random_id = user_ids[0]
    while random_id in user_ids_set:
        random_id = int(round(hist_dist.rvs(size=1)[0]))
    random_ids.append(random_id)
    
print(len(random_ids))
randomized_df["user_id"] = pd.Series(np.asarray(random_ids).astype(int))
print(randomized_df.head())

19908
  university              user_id       user_name  number_tweets  \
0  UFrontera     1732490562035142         rsMKpJv             30   
1      usach  1180791938325468928  ombredrNoticia          20863   
2  UFrontera   911825445201056384     k6KrisMiEoo             28   
3    pucv_cl    10437195470316396        QrREp9el            249   
4    pucv_cl     3856187579930749    JMdibelchj12             25   

   friends_count  followers_count                      created_at  
0            107                2  Tue Mar 03 01:14:09 +0000 2020  
1           1917              957  Wed Oct 02 17:43:58 +0000 2013  
2            210               35  Thu Mar 24 16:20:54 +0000 2016  
3            127               21  Mon Sep 05 22:49:11 +0000 2011  
4             51                8  Sun May 31 19:58:21 +0000 2015  


In [10]:
# Shuffleamos por si acaso (???) 
randomized_df = randomized_df.sample(frac=1).reset_index(drop=True)
randomized_df.to_csv("universities_followers_no_dups_random.csv", index = False)